In [2]:
# All imports
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import plotly.express as px
from CRUD import AnimalShelter

# Login Credentials
username = "aacuser"
password = "momosushi2013"
host = 'nv-desktop-services.apporto.com'
port = 31543
db_name = 'aac'
collection_name = 'animals'

db = AnimalShelter(username, password, host, port, db_name, collection_name)
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

app = JupyterDash(__name__)

image_filename = 'grazsal.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Div(className='row', style={'textAlign': 'center', 'alignItems': 'center', 'justifyContent': 'center'}, children=[
        html.Div(className='col s12', children=[
            html.H1('SNHU CS-340 Dashboard'),
            html.H2('Faizah Mas Mohd Khalik'),
            html.A(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'100px', 'width':'auto'}), href='http://www.snhu.edu', target='_blank'),
        ]),
    ]),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DRIT'},
            {'label': 'Reset', 'value': 'RESET'},
        ],
        value='RESET',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '300px', 'overflowY': 'auto'}
    ),
    html.Div(className='row', style={'display': 'flex', 'marginTop': '20px'}, children=[
        html.Div(id='map-id', style={'width': '50%', 'height': '500px'}, className='col s12 m6'),
        html.Div(id='graph-id', style={'width': '50%'}, className='col s12 m6'),
    ]),
])

# The Filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'WR':
        df_filtered = df[(df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) & (df['sex_upon_outcome'] == 'Intact Female') & (df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'MWR':
        df_filtered = df[(df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) & (df['sex_upon_outcome'] == 'Intact Male') & (df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'DRIT':
        df_filtered = df[(df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) & (df['sex_upon_outcome'] == 'Intact Male') & (df['age_upon_outcome_in_weeks'] >= 20) & (df['age_upon_outcome_in_weeks'] <= 300)]
    else:
        df_filtered = df
    return df_filtered.to_dict('records')

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_map(data):
    dff = pd.DataFrame(data)
    markers = [dl.Marker(position=[row['location_lat'], row['location_long']]) for index, row in dff.iterrows()]
    return [dl.Map(center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()] + markers)]

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_graph(data):
    dff = pd.DataFrame(data)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)

app.run_server(debug=True)



Dash app running on http://127.0.0.1:13265/
